# Simple imaging with WebbPSF

Creating JWST PSFs documentation notbeook: https://github.com/spacetelescope/webbpsf/blob/develop/notebooks/WebbPSF_tutorial.ipynb

Simulated OTE moves (aberrating segments): https://github.com/spacetelescope/webbpsf/blob/develop/notebooks/Simulated%20OTE%20Mirror%20Move%20Demo.ipynb

All info about JWST imaging here:
https://webbpsf.readthedocs.io/en/latest/jwst.html#

NIRCam coronagraphy recommended strategies: https://jwst-docs.stsci.edu/near-infrared-camera/nircam-observing-strategies/nircam-coronagraphic-imaging-recommended-strategies

JWST Inner Working Angles: https://jwst-docs.stsci.edu/methods-and-roadmaps/jwst-high-contrast-imaging/hci-supporting-technical-information/hci-inner-working-angle

JWST High Contrast Imaging: https://jwst-docs.stsci.edu/methods-and-roadmaps/jwst-high-contrast-imaging

In [ ]:
import os
import astropy.units as u
import matplotlib
%matplotlib inline
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
import numpy as np
import webbpsf

from pastis.config import CONFIG_PASTIS
from pastis.e2e_simulators import webbpsf_imaging
import pastis.util

## Display simple PSF after changing individual parameters in succession 

### Make NIRCam object and display optical train and PSF

In [ ]:
nc = webbpsf.NIRCam()
nc.include_si_wfe = False
nc, ote = webbpsf.enable_adjustable_ote(nc)
ote.zero()

In [ ]:
plt.figure(figsize=(20,10))
nc_psf = nc.calc_psf(nlambda=1, display=True)

In [ ]:
# Display just PSF
webbpsf.display_psf(nc_psf)

### Set the filter

In [ ]:
nc.filter = CONFIG_PASTIS.get('JWST', 'filter_name')
plt.figure(figsize=(20,10))
nc_filter = nc.calc_psf(nlambda=1, display=True)

In [ ]:
# Display just PSF
webbpsf.display_psf(nc_filter)

### Set the Lyot stop (pupil mask)

In [ ]:
nc.pupil_mask = CONFIG_PASTIS.get('JWST', 'pupil_plane_stop')
plt.figure(figsize=(20,10))
nc_pupil_mask = nc.calc_psf(nlambda=1, display=True)

In [ ]:
# Display just PSF
webbpsf.display_psf(nc_pupil_mask)

### Set the FPM (image mask)

In [ ]:
nc.image_mask = CONFIG_PASTIS.get('JWST', 'focal_plane_mask')
plt.figure(figsize=(20,10))
nc_image_mask = nc.calc_psf(nlambda=1, display=True)

In [ ]:
# Display just PSF
webbpsf.display_psf(nc_image_mask, vmin=1e-12, vmax=1e-6)

## Get NIRCam object and its OTE form PASTIS E2E function

In [ ]:
jwst = webbpsf_imaging.set_up_nircam()    # this returns a tuple of two: jwst[0] is the nircam object, jwst[1] its ote

In [ ]:
plt.figure(figsize=(19, 19))
jwst[0].display()

### Make a direct PSF

In [ ]:
plt.figure(figsize=(20,10))
direct = jwst[0].calc_psf(nlambda=1, display=True)

In [ ]:
plt.figure(figsize=(10,10))
webbpsf.display_psf(direct)

In [ ]:
direct_psf = direct[0].data
plt.figure(figsize=(10,10))
plt.imshow(direct_psf, norm=LogNorm())

In [ ]:
norm = direct_psf.max()
print(norm)

### Make a coronagraphic PSF

In [ ]:
jwst[0].image_mask = CONFIG_PASTIS.get('JWST', 'focal_plane_mask')
plt.figure(figsize=(20,10))
coro_image = jwst[0].calc_psf(nlambda=1, display=True)

In [ ]:
coro_psf = coro_image[0].data / norm
plt.figure(figsize=(10,10))
plt.imshow(direct_psf, norm=LogNorm())
plt.colorbar()

### Create dark hole mask

In [ ]:
iwa = CONFIG_PASTIS.getfloat('JWST', 'IWA')
owa = CONFIG_PASTIS.getfloat('JWST', 'OWA')
sampling = 8#CONFIG_PASTIS.getfloat('JWST', 'sampling')
dh_mask = pastis.util.create_dark_hole(coro_psf, iwa, owa, sampling).astype('bool')

plt.figure(figsize=(20,10))
plt.subplot(1, 2, 1)
plt.imshow(dh_mask)
plt.subplot(1, 2, 2)
plt.imshow(coro_psf * dh_mask)

In [ ]:
# Measure average contrast in dark hole
contrast_floor = pastis.util.dh_mean(coro_psf, dh_mask)
print(contrast_floor)